# Graph Neural Network

### Loading the files

In [1]:
import pandas as pd
nodes_df = pd.read_csv('Raw Dataset/Homogenoeus/Nodes/NodesIndex.csv')
nodes_Plant_Storage_df = pd.read_csv('Raw Dataset/Homogenoeus/Nodes/Nodes Type (Plant & Storage).csv')

edges_plant_df = pd.read_csv('Raw Dataset/Homogenoeus/Edges/EdgesIndex/Edges (Plant).csv')
edges_group_df = pd.read_csv('Raw Dataset/Homogenoeus/Edges/EdgesIndex/Edges (Product Group).csv')
edges_subgroup_df = pd.read_csv('Raw Dataset/Homogenoeus/Edges/EdgesIndex/Edges (Product Sub-Group).csv')
edges_storage_df = pd.read_csv('Raw Dataset/Homogenoeus/Edges/EdgesIndex/Edges (Storage Location).csv')

In [2]:
print('nodes_df')
print(nodes_df.head(2))
print('nodes_Plant_Storage_df')
print(nodes_Plant_Storage_df.head(2))
print('edges_plant_df')
print(edges_plant_df.head(2))
print('edges_group_df')
print(edges_group_df.head(2))
print('edges_subgroup_df')
print(edges_subgroup_df.head(2))
print('edges_storage_df')
print(edges_storage_df.head(2))

nodes_df
         Node  NodeIndex
0  SOS008L02P          0
1  SOS005L04P          1
nodes_Plant_Storage_df
         Node  Plant  Storage Location
0  POV002L09P   2120            2030.0
1  SOS001L12P   2120            2030.0
edges_plant_df
   Plant  node1  node2
0   1901     28     24
1   1903     22     24
edges_group_df
   node1  node2 GroupCode
0      0      1         S
1      0      2         S
edges_subgroup_df
   node1  node2 SubGroupCode
0      0      1          SOS
1      0      2          SOS
edges_storage_df
   Storage Location  node1  node2
0             130.0     28     24
1             330.0      1      3


## 1. Creating the graph object

We need to convert our data in the format of several csv files into a graph structure in order to apply GNNs.

In [3]:
import networkx as nx
import torch
from torch_geometric.utils import from_networkx

In [4]:
G = nx.Graph()

## Nodes
# Products
for index, row in nodes_df.iterrows():
    G.add_node(row['NodeIndex'], name=row['NodeIndex'], plant = nodes_Plant_Storage_df[nodes_Plant_Storage_df['Node'] == row['Node']]['Plant'].values[0], storage = nodes_Plant_Storage_df[nodes_Plant_Storage_df['Node'] == row['Node']]['Storage Location'].values[0])


## Edges
# Groups - Subgroups
for index, row in edges_plant_df.iterrows():
    G.add_edge(row['node1'], row['node2'], Plant=row['Plant'])
for index, row in edges_group_df.iterrows():
    G.add_edge(row['node1'], row['node2'], Group=row['GroupCode'])
for index, row in edges_subgroup_df.iterrows():
    G.add_edge(row['node1'], row['node2'], SubGroup=row['SubGroupCode'])
for index, row in edges_storage_df.iterrows():
    G.add_edge(row['node1'], row['node2'], Storage=row['Storage Location'])



#### Visualizing the graph

In [5]:
import plotly.graph_objects as go


# Extract node positions
pos = nx.spring_layout(G)

# Create edge traces
edge_x = []
edge_y = []
for edge in G.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_x.append(x0)
    edge_x.append(x1)
    edge_x.append(None)
    edge_y.append(y0)
    edge_y.append(y1)
    edge_y.append(None)

edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines')

# Create node traces
node_x = []
node_y = []
for node in G.nodes():
    x, y = pos[node]
    node_x.append(x)
    node_y.append(y)

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        colorscale='YlGnBu',
        size=10,
        colorbar=dict(
            thickness=15,
            title='Node Connections',
            xanchor='left',
            titleside='right'
        ),
        line_width=2))

# Create Plotly figure
fig = go.Figure(data=[edge_trace, node_trace],
                layout=go.Layout(
                    title='Network graph',
                    showlegend=False,
                    hovermode='closest',
                    margin=dict(b=0,l=0,r=0,t=40),
                    xaxis=dict(showgrid=False, zeroline=False),
                    yaxis=dict(showgrid=False, zeroline=False))
                )

fig.show()

## 2.Building the GNN

## 3.Evaluation